### Decision Trees and Ensemble Models in classification and regression problems.

## Learning outcomes 

- Understand how to use decision trees on a Dataset to make a prediction
- Learning hyper-parameters tuning for decision trees by using RandomGrid 
- Learning the effectiveness of ensemble algorithms (Random Forest, Adaboost, Extra trees classifier, Gradient Boosted Tree)

I will use Classification Trees for predicting if a user has a default payment option active or not. 

This dataset is aimed at the case of customer default payments in Taiwan. From the perspective of risk management, the result of predictive accuracy of the estimated probability of default will be more valuable than the binary result of classification - credible or not credible clients. Because the real probability of default is unknown, this study presented the novel Sorting Smoothing Method to estimate the real probability of default.


In [ ]:
#required imports
import numpy as np
import pandas as pd
from numpy import bincount, linspace, mean, std, arange, squeeze
import itertools, time, datetime
from scipy.stats import randint
from pandas import set_option
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import graphviz 
from graphviz import Source
from IPython.display import SVG

from sklearn import tree
#from sklearn.tree.export import export_text
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn import metrics
from sklearn.metrics import f1_score,confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve, precision_score, recall_score


After installing the necessary libraries, proceed to download the data. Since reading the excel file won't create headers by default, we added two more operations to substitute the columns.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
#loading the data
df = pd.read_csv("/kaggle/input/default-of-credit-card-clients-dataset/UCI_Credit_Card.csv")
#df.columns = df.iloc[0]
df.drop(['ID'], inplace=True,axis = 1)

In [ ]:
pd.set_option('display.max_columns', None)

In the following, you can take a look into the dataset.

In [ ]:
df.head(10)

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
df.rename(columns={"PAY_0":"PAY_1", "default.payment.next.month": "DEFAULT"}, inplace = True)

In [ ]:
print(df.EDUCATION.unique())
print(df.MARRIAGE.unique())

In [ ]:
fil = (df['EDUCATION'] == 5) | (df['EDUCATION'] == 6) | (df['EDUCATION'] == 0)
df.loc[fil, 'EDUCATION'] = 4
df['EDUCATION'].value_counts()

In [ ]:
fil = (df['MARRIAGE'] == 0)
df.loc[fil, 'MARRIAGE'] = 3
df['MARRIAGE'].value_counts()

In [ ]:
df['AGE'].unique()

In [ ]:
## Creating a Function to Distribute the Age
def func(x):
    if(x >=20 and x<30 ):
        return 1
    elif(x>=30 and x<40):
        return 2
    elif(x>=40 and x<50):
        return 3
    elif(x>=50 and x<60):
        return 4
    elif(x>=60 and x<=80):
        return 5

In [ ]:
df['AGE'] = df['AGE'].apply(func)

In [ ]:
X=df[df.columns[:-1]]
y=df['DEFAULT']

## Questions (15 points total)

#### Question 1 (2 pts)
Build a classifier by using decision tree and calculate the confusion matrix. Try different hyper-parameters (at least two) and discuss the result.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25)

In [ ]:
# Standardization
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### DECISION TREE CLASSIFIER

In [ ]:
tr = tree.DecisionTreeClassifier(max_depth=3, criterion='gini', random_state=25)

# Train the estimator.
tr.fit(X_train, y_train)

### LETS PLOT A TREE

In [ ]:
dot_data = tree.export_graphviz(tr, out_file=None, feature_names=X.columns, filled=True, rounded=True, special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

In [ ]:
# Make predictions.
tr_pred=tr.predict(X_test)
print(tr_pred)

# CV score
tr_cv=cross_val_score(tr, X_train, y_train, cv=10).mean()
print ("Cross val: %.3f" % tr_cv)

In [ ]:
# Accuracy: 1 is perfect prediction.
print('Accuracy: %.3f' % tr.score(X_test, y_test))

# Precision
print('Precision: %.3f' % precision_score(y_test, tr_pred))

# Recall
print('Recall: %.3f' % recall_score(y_test, tr_pred))

# f1 score: best value at 1 (perfect precision and recall) and worst at 0.
print('F1 score: %.3f' % f1_score(y_test, tr_pred))

In [ ]:
#### CONFUSION MATRIX FOR DECISION TREE

In [ ]:
# Plot confusion matrix for Decision tree.
tr_matrix = confusion_matrix(y_test,tr_pred)
sns.set(font_scale=1.3)
plt.subplots(figsize=(8, 8))
sns.heatmap(tr_matrix,annot=True, cbar=False, cmap='twilight',linewidth=0.5,fmt="d")
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix for Decision tree');

In [ ]:
# Predict propabilities for the test data.
tr_probs = tr.predict_proba(X_test)

# Keep Probabilities of the positive class only.
tr_probs = tr_probs[:, 1]

# Compute the AUC Score.
auc_tr = roc_auc_score(y_test, tr_probs)
print('AUC: %.2f' % auc_tr)

### USING GRID SEARCH TO WORK ON HYPER - PARAMETERS

In [ ]:
parameters = {'criterion':['gini','entropy'],
              'max_depth':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
             }

# GridSearchCV estimator.
gs_tree = GridSearchCV(tr, parameters, cv=10, n_jobs=-1,verbose=1)

# Train the GridSearchCV estimator and search for the best parameters.
gs_tree.fit(X_train,y_train)

In [ ]:
# Make predictions with the best parameters.
gs_tree_pred=gs_tree.predict(X_test)

In [ ]:
# Best parameters.
print("Best Decision tree Parameters: {}".format(gs_tree.best_params_))

# Cross validation accuracy for the best parameters.
print('Cross-validation accuracy: %0.3f' % gs_tree.best_score_)

# Accuracy: 1 is perfect prediction.
print('Accuracy: %0.3f' % (gs_tree.score(X_test,y_test)))

# Precision
print('Precision: %.3f' % precision_score(y_test, gs_tree_pred))

# Recall
print('Recall: %.3f' % recall_score(y_test, gs_tree_pred))

# f1 score: best value at 1 (perfect precision and recall) and worst at 0.
print('F1 score: %.3f' % f1_score(y_test, gs_tree_pred))

In [ ]:
# Plot confusion matrix for Decision tree.
gs_tr_matrix = confusion_matrix(y_test,gs_tree_pred)
sns.set(font_scale=1.3)
plt.subplots(figsize=(8, 8))
sns.heatmap(gs_tr_matrix,annot=True, cbar=False, cmap='twilight',linewidth=0.5,fmt="d")
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix for GridSearchCV Decision tree');

In [ ]:
# Predict probabilities for the test data.
gs_tree_probs = gs_tree.predict_proba(X_test)

# Keep Probabilities of the positive class only.
gs_tree_probs = gs_tree_probs[:, 1]

# Compute the AUC Score.
gs_tree_auc = roc_auc_score(y_test, gs_tree_probs)
print('AUC: %.2f' % gs_tree_auc)

## LETS SEE THEM ON ROC CURVE

In [ ]:
# Get the ROC Curves.
gs_tr_fpr, gs_tr_tpr, gs_tr_thresholds = roc_curve(y_test, gs_tree_probs)
tr_fpr, tr_tpr, tr_thresholds = roc_curve(y_test, tr_probs)

# Plot the ROC curves.
plt.figure(figsize=(8,8))
plt.plot(tr_fpr, tr_tpr, color='red', label='Decision tree ROC (AUC= %0.2f)'% auc_tr)
plt.plot(gs_tr_fpr, gs_tr_tpr, color='green', label='GridSearch+Decision tree ROC (AUC= %0.2f)'% gs_tree_auc)
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--',label='random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend()
plt.show()

### We can see that GRID SEARCH dint work well and we are ahving better result with only DECISION TREE . We have AUC as 0.74 with DECISON TREE whereas it dropped to 0.64 when we applied GRID SEARCH.

In [ ]:
metrics=['Accuracy', 'CV accuracy', 'Precision','Recall','F1','ROC AUC']

fig = go.Figure(data=[
    go.Bar(name='Decision tree', x=metrics,
           y=[tr.score(X_test, y_test),tr_cv,precision_score(y_test, tr_pred),recall_score(y_test, tr_pred),f1_score(y_test, tr_pred),auc_tr]),
    
    go.Bar(name='GridSearchCV+Decision tree',
           x=metrics, y=[gs_tree.score(X_test,y_test),gs_tree.best_score_,precision_score(y_test, gs_tree_pred),recall_score(y_test, gs_tree_pred), f1_score(y_test, gs_tree_pred),gs_tree_auc]),
])

fig.update_layout(title_text='Metrics for each model',
                  barmode='group',xaxis_tickangle=-45,bargroupgap=0.05)
fig.show()

#### Question 2 (4 pts)

Try to build the decision tree which you built for the previous question, but this time by RandomizedSearchCV over hyper-parameters. Compare the results.

In [ ]:
from scipy.stats import randint

from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_dist = {"max_depth": [3, None],
              "max_features": randint(1, 9),
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]}


In [ ]:
random_search_cv = RandomizedSearchCV(tr, param_dist, cv=5)

In [ ]:
random_search_cv.fit(X_train,y_train)

In [ ]:
# Make predictions with the best parameters.
random_search_cv_pred=random_search_cv.predict(X_test)

In [ ]:
# Best parameters.
print("Best Decision tree Parameters: {}".format(random_search_cv.best_params_))

# Cross validation accuracy for the best parameters.
print('Cross-validation accuracy: %0.3f' % random_search_cv.best_score_)

# Accuracy: 1 is perfect prediction.
print('Accuracy: %0.3f' % (random_search_cv.score(X_test,y_test)))

# Precision
print('Precision: %.3f' % precision_score(y_test, random_search_cv_pred))

# Recall
print('Recall: %.3f' % recall_score(y_test, random_search_cv_pred))

# f1 score: best value at 1 (perfect precision and recall) and worst at 0.
print('F1 score: %.3f' % f1_score(y_test, random_search_cv_pred))

In [ ]:
# Plot confusion matrix for Decision tree.
random_tr_matrix = confusion_matrix(y_test,random_search_cv_pred)
sns.set(font_scale=1.3)
plt.subplots(figsize=(8, 8))
sns.heatmap(random_tr_matrix,annot=True, cbar=False, cmap='twilight',linewidth=0.5,fmt="d")
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix for RandomSearchCV Decision tree');

In [ ]:
# Predict probabilities for the test data.
random_search_cv_probs = random_search_cv.predict_proba(X_test)

# Keep Probabilities of the positive class only.
random_search_cv_probs = random_search_cv_probs[:, 1]

# Compute the AUC Score.
random_search_cv_auc = roc_auc_score(y_test, random_search_cv_probs)
print('AUC: %.2f' % random_search_cv_auc)

In [ ]:
# Get the ROC Curves.
random_tr_fpr, random_tr_tpr, random_tr_thresholds = roc_curve(y_test, random_search_cv_probs)
gs_tr_fpr, gs_tr_tpr, gs_tr_thresholds = roc_curve(y_test, gs_tree_probs)
tr_fpr, tr_tpr, tr_thresholds = roc_curve(y_test, tr_probs)

# Plot the ROC curves.
plt.figure(figsize=(8,8))
plt.plot(tr_fpr, tr_tpr, color='red', label='Decision tree ROC (AUC= %0.2f)'% auc_tr)
plt.plot(gs_tr_fpr, gs_tr_tpr, color='green', label='GridSearch+Decision tree ROC (AUC= %0.2f)'% gs_tree_auc)
plt.plot(random_tr_fpr, random_tr_tpr, color='yellow', label='RandomSearchCV+Decision tree ROC (AUC= %0.2f)'% random_search_cv_auc)
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--',label='random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend()
plt.show()

In [ ]:
metrics=['Accuracy', 'CV accuracy', 'Precision','Recall','F1','ROC AUC']

fig = go.Figure(data=[
    go.Bar(name='Decision tree', x=metrics,
           y=[tr.score(X_test, y_test),tr_cv,precision_score(y_test, tr_pred),recall_score(y_test, tr_pred),f1_score(y_test, tr_pred),auc_tr]),
    
    go.Bar(name='GridSearchCV+Decision tree',
           x=metrics, y=[gs_tree.score(X_test,y_test),gs_tree.best_score_,precision_score(y_test, gs_tree_pred),recall_score(y_test, gs_tree_pred), f1_score(y_test, gs_tree_pred),gs_tree_auc]),

    go.Bar(name='RandomSearchCV+Decision tree',
           x=metrics, y=[random_search_cv.score(X_test,y_test),random_search_cv.best_score_,precision_score(y_test, random_search_cv_pred),recall_score(y_test, random_search_cv_pred), f1_score(y_test, random_search_cv_pred),random_search_cv_auc]),
])

fig.update_layout(title_text='Metrics for each model',
                  barmode='group',xaxis_tickangle=-45,bargroupgap=0.05)
fig.show()

In [ ]:
d={
'': ['Decision Tree','GridSearchCV + Decision Tree','RandomSearchCV + Decision Tree'],
'Accuracy': [tr.score(X_test, y_test),gs_tree.score(X_test,y_test),random_search_cv.score(X_test, y_test)],
'CV Accuracy': [tr_cv,gs_tree.best_score_,random_search_cv.best_score_],
'Precision': [precision_score(y_test, tr_pred),precision_score(y_test, gs_tree_pred),precision_score(y_test, random_search_cv_pred)],
'Recall': [recall_score(y_test, tr_pred),recall_score(y_test, gs_tree_pred),recall_score(y_test, random_search_cv_pred)],
'F1': [f1_score(y_test, tr_pred),f1_score(y_test, gs_tree_pred),f1_score(y_test, random_search_cv_pred)],
'ROC AUC': [auc_tr, gs_tree_auc, random_search_cv_auc]
}

results=pd.DataFrame(data=d).round(3).set_index('')
results

### As we see the comparison RANDOMIZED SEARCH gave us better result than GRID SEARCH but they are similar to normal DECISION TREE.

In [ ]:
random_search_cv.best_estimator_

In [ ]:
print("Tuned Decision Tree Parameters: {}".format(random_search_cv.best_params_))
print("Best score is {}".format(random_search_cv.best_score_))

#### Question 3 (6 pts)

Try to build the same classifier by using following ensemble models. For each of these models calculate accuracy and at least for two in the list below, plot the learning curves.

* Random Forest 
* AdaBoost
* Extra Trees Classifier 
* Gradient Boosted Trees 


In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from time import time

### AdaBoost Classifier

In [ ]:
ada_clf = AdaBoostClassifier(
    tr, n_estimators=50,
    algorithm="SAMME.R", learning_rate=0.5)
start = time()
ada_clf.fit(X_train, y_train)
end = time()
train_time_ada=end-start


In [ ]:
# Predict probabilities for the test data.
ada_clf_probs = ada_clf.predict_proba(X_test)

# Keep Probabilities of the positive class only.
ada_clf_probs = ada_clf_probs[:, 1]

# Compute the AUC Score.
ada_clf_auc = roc_auc_score(y_test, ada_clf_probs)
print('AUC: %.2f' % ada_clf_auc)

In [ ]:
y_pred = ada_clf.predict(X_test)
#from sklearn.metrics import  accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
roc=roc_auc_score(y_test, ada_clf_probs)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results = pd.DataFrame([['Adaboost', acc,prec,rec, f1,roc]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score','ROC'])
results

### Extra Tree Classifier

In [ ]:
forest = ExtraTreesClassifier(n_estimators=50)
start = time()
forest.fit(X_train, y_train)
end = time()
train_time_et=end-start


In [ ]:
# Predict probabilities for the test data.
forest_probs = forest.predict_proba(X_test)

# Keep Probabilities of the positive class only.
forest_probs = forest_probs[:, 1]

# Compute the AUC Score.
forest_auc = roc_auc_score(y_test, forest_probs)
print('AUC: %.2f' % forest_auc)

In [ ]:
y_pred = forest.predict(X_test)
#from sklearn.metrics import  accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
roc=roc_auc_score(y_test, forest_probs)
acc = accuracy_score(y_test, y_pred)
prec= precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['Extra Tree ', acc,prec,rec, f1,roc]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score','ROC'])
results = results.append(model_results, ignore_index = True)
results

### Random Forest Classifier

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=50, max_leaf_nodes=16, n_jobs=-1)
start = time()
rnd_clf.fit(X_train, y_train)
end=time()
train_time_r50=end-start

In [ ]:
# Predict probabilities for the test data.
rnd_clf_probs = rnd_clf.predict_proba(X_test)

# Keep Probabilities of the positive class only.
rnd_clf_probs = rnd_clf_probs[:, 1]

# Compute the AUC Score.
rnd_clf_auc = roc_auc_score(y_test, rnd_clf_probs)
print('AUC: %.2f' % rnd_clf_auc)

In [ ]:
y_pred_r = rnd_clf.predict(X_test)
#from sklearn.metrics import  accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
roc=roc_auc_score(y_test, rnd_clf_probs)
acc = accuracy_score(y_test, y_pred_r)
prec = precision_score(y_test, y_pred_r)
rec = recall_score(y_test, y_pred_r)
f1 = f1_score(y_test, y_pred_r)

model_results = pd.DataFrame([['Random_forest ', acc,prec,rec, f1,roc]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score','ROC'])
results = results.append(model_results, ignore_index = True)
results

### Gradient Boosting Classifier

In [ ]:
gbes = GradientBoostingClassifier(n_estimators=50)
start = time()

end=time()
train_time_g=end-start
gbes.fit(X_train, y_train)
end=time()
train_time_g=end-start


In [ ]:
y_pred = gbes.predict(X_test)

roc=roc_auc_score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['Gboost', acc,prec,rec, f1,roc]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score','ROC'])
results = results.append(model_results,ignore_index=True)
results

In [ ]:
# Predict probabilities for the test data.
gbes_probs = gbes.predict(X_test)

# Keep Probabilities of the positive class only.
#gbes_probs = gbes_probs[:, 1]

# Compute the AUC Score.
gbes_auc = roc_auc_score(y_test, gbes_probs)
print('AUC: %.2f' % gbes_auc)

### As we can see after using all the four ensemblers , we were able to get better results than our DECISION TREE MODEL. Out of the four Ensemblers RANDOM FOREST did best with AUC as 0.78.
### But if we look at the other factors like ACCURACY, PRECISION, RECALL, F1 SCORE then GRADIENT BOOSTING is better than all of them.

In [ ]:
model = ['Adaboost','ExtraTree','Random forest', 'GBOOST']
Train_Time = [
    train_time_ada,
    train_time_et,
    train_time_r50,
    train_time_g

]
index = np.arange(len(model))
plt.bar(index, Train_Time)
plt.xlabel('Machine Learning Models', fontsize=15)
plt.ylabel('Training Time', fontsize=15)
plt.xticks(index, model, fontsize=8, )
plt.title('Comparison of Training Time of all ML models')
plt.show()

### If we compare looking at the time they toom then RANDOM FOREST is the fastest.

In [ ]:
# Get the ROC Curves.
rnd_clf_fpr, rnd_clf_tpr, rnd_clf_thresholds = roc_curve(y_test, rnd_clf_probs)
ada_clf_fpr, ada_clf_tpr, ada_clf_thresholds = roc_curve(y_test, ada_clf_probs)
forest_fpr, forest_tpr, forest_thresholds = roc_curve(y_test, forest_probs)
gbes_fpr, gbes_tpr, gbes_thresholds = roc_curve(y_test, gbes_probs)

# Plot the ROC curves.
plt.figure(figsize=(8,8))
plt.plot(ada_clf_fpr, ada_clf_tpr, color='red', label='AdaBoost ROC (AUC= %0.2f)'% ada_clf_auc)
plt.plot(rnd_clf_fpr, rnd_clf_tpr, color='green', label='RandomForest ROC (AUC= %0.2f)'% rnd_clf_auc)
plt.plot(forest_fpr, forest_tpr, color='black', label='ExtraTreeClassifier ROC (AUC= %0.2f)'% forest_auc)
plt.plot(gbes_fpr, gbes_tpr, color='yellow', label='GradientBoosting ROC (AUC= %0.2f)'% gbes_auc)
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--',label='random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend()
plt.show()

### If we look at the ROC curve we can see that Random Forest is performing best out of the four ensemblers.

In [ ]:
from sklearn.model_selection import learning_curve
train_sizes, train_scores, test_scores = learning_curve(estimator=rnd_clf, X=X_train, y=y_train,
                                                       cv=10, train_sizes=np.linspace(0.1, 1.0, 10),
                                                     n_jobs=1)
#
# Calculate training and test mean and std
#
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
#
# Plot the learning curve
#
plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean, color='green', marker='+', markersize=5, linestyle='--', label='Validation Accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.title('Random Forest Learning Curve ')
plt.xlabel('Training Data Size')
plt.ylabel('Model accuracy')
plt.grid()
plt.legend(loc='lower right')
plt.show()


In [ ]:
train_sizes, train_scores, test_scores = learning_curve(estimator=ada_clf, X=X_train, y=y_train,
                                                       cv=10, train_sizes=np.linspace(0.1, 1.0, 10),
                                                     n_jobs=1)
#
# Calculate training and test mean and std
#
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
#
# Plot the learning curve
#
plt.plot(train_sizes, train_mean, color='red', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='red')
plt.plot(train_sizes, test_mean, color='black', marker='+', markersize=5, linestyle='--', label='Validation Accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='black')
plt.title('AdaBoost Learning Curve ')
plt.xlabel('Training Data Size')
plt.ylabel('Model accuracy')
plt.grid()
plt.legend(loc='lower right')
plt.show()

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(estimator=forest, X=X_train, y=y_train,
                                                       cv=10, train_sizes=np.linspace(0.1, 1.0, 10),
                                                     n_jobs=1)
#
# Calculate training and test mean and std
#
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
#
# Plot the learning curve
#
plt.plot(train_sizes, train_mean, color='red', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='red')
plt.plot(train_sizes, test_mean, color='black', marker='+', markersize=5, linestyle='--', label='Validation Accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='black')
plt.title('ExtraTreeClassifier Learning Curve ')
plt.xlabel('Training Data Size')
plt.ylabel('Model accuracy')
plt.grid()
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(gbes, 
                                                        X, 
                                                        y,
                                                        # Number of folds in cross-validation
                                                        cv=10,
                                                        # Evaluation metric
                                                        scoring='accuracy',
                                                        # Use all computer cores
                                                        n_jobs=-1, 
                                                        # 50 different sizes of the training set
                                                        train_sizes=np.linspace(0.01, 1.0, 50))

# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Draw lines
plt.plot(train_sizes, train_mean, '--', color="red",  label="Training score")
plt.plot(train_sizes, test_mean, color="black", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Gradient Boosting Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

### Often, we are not aware of optimal values for hyperparameters which would generate the best model output. So, what we tell the model is to explore and select the optimal model architecture automatically. 
### Each model has its own sets of parameters that need to be tuned to get optimal output. For every model, our goal is to minimize the error or say to have predictions as close as possible to actual values.



### RANDOMIZED SEARCH
The model randomly makes combinations of its own and tries to fit the dataset and test the accuracy. Here, chances are there to miss on a few combinations which could have been optimal ones. Although, random search consumes quite less amount of time and most of the time it gives optimal solutions as well.

### GRID SEARCH
In this method, each combination of hyperparameter value is tried. This makes the process time consuming, or in short, inefficient. This method is quite an expensive method in terms of computation power and time, but this is the most efficient method as there is the least possibility of missing out on an optimal solution for a model.

### As we saw above our model performance got low when we used GRID SEARCH but improved when we used RANDOMIZED SEARCH CV. SO changing hyperparameters did make a little difference to our model performance.

### There are a number of machine learning models to choose from. When we build these models, we always use a set of historical data to help our machine learning algorithms learn what is the relationship between a set of input features to a predicted output. Below are few reasons why the model dint work or performed better:


## High Bias or High Variance

### High Bias refers to a scenario where your model is “underfitting” your dataset . This is bad because your model is not presenting a very accurate or representative picture of the relationship between your inputs and predicted output, and is often outputting high error .

### In cases of High Variance or “overfitting”, your machine learning model is so accurate that it is perfectly fitted to your example dataset. While this may seem like a good outcome, it is also a cause for concern, as such models often fail to generalize to future datasets. So while your model works well for your existing data, you don’t know how well it’ll perform on other examples.

### If you can generate a model with overall low error in both your train (past) and test (future) datasets, you’ll have found a model that is “Just Right” and balanced the right levels of bias and variance.


## Low Precision or Low Recall

### Even when you have high accuracy, it’s possible that your machine learning model may be susceptible to other types of error.

### Precision is a measure of how often your predictions for the positive class are actually true. It’s calculated as the number of True Positives over the sum of the True Positives and False Positives.

### Recall is the measure of how often the actual positive class is predicted as such. It’s calculated as the number of True Positives over the sum of the True Positives and False Negatives.

### Another way to interpret the difference between Precision and Recall, is that Precision is measuring what fraction of your predictions for the positive class are valid, while Recall is telling you how often your predictions actually capture the positive class. Hence, a situation of Low Precision emerges when very few of your positive predictions are true, and Low Recall occurs if most of your positive values are never predicted.

### The goal of a good machine learning model is to get the right balance of Precision and Recall, by trying to maximize the number of True Positives while minimizing the number of False Negatives and False Positives 

### The above probelm can be solved by doing following changes:

####    Incase of High Bias - increasing the number of input features.
####    Incase of High Variance - reduce the number of input features. Increasing the number of training examples.
####    Incase of Low Precision - increase the probability threshold
####    Incase of Low Recall - reduce the probability threshold, therein predicting the positive class more often.

### Now we know that our data was imbalanced and need resampling with feature engineering thats why models were not able to perform as expected.